In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Embedding, Flatten, Dot
from tensorflow.keras.models import Sequential,Model
from keras.layers import Input
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_excel('datadummy2.xlsx')
df.head()

,Id User,Genre,Nama Usaha,Lokasi,Rating,Nama Produk,Jumlah Follower
0,1,Baju,Fashion Trend,Jakarta,4.2,Kemeja Casual,3500.0
1,2,Makanan,Warung Sate,Tangerang,4.5,Sate Ayam Madura,4600.0
2,3,Elektronik,Gadget Store,Bogor,4.1,Headphone Bluetooth,3800.0
3,4,Aksesori,Accessorize It,Bekasi,4.3,Gelang Kalung Set,2500.0
4,5,Kecantikan,Beauty Paradise,Depok,4.4,Lipstik Matte,3200.0


Cleaning The duplicated data

In [3]:
df['Nama Usaha'].duplicated()
df = df.drop_duplicates(subset=['Nama Usaha'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 902
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id User          730 non-null    int64  
 1   Genre            730 non-null    object 
 2   Nama Usaha       730 non-null    object 
 3   Lokasi           730 non-null    object 
 4   Rating           730 non-null    float64
 5   Nama Produk      730 non-null    object 
 6   Jumlah Follower  728 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 45.6+ KB


Checking the null in the data

In [4]:
print(df.isnull().sum())

Id User            0
Genre              0
Nama Usaha         0
Lokasi             0
Rating             0
Nama Produk        0
Jumlah Follower    2
dtype: int64


Replace the null value with 0

In [5]:
# Replace missing values with 0 in 'Jumlah Follower' column
df['Jumlah Follower'] = df['Jumlah Follower'].fillna(0)

# Print the updated DataFrame
print(df.isnull().sum())

Id User            0
Genre              0
Nama Usaha         0
Lokasi             0
Rating             0
Nama Produk        0
Jumlah Follower    0
dtype: int64


one hot encoding the string value from the column

In [6]:
# Encode genre using one-hot encoding
encoder = OneHotEncoder()
genre_encoded = encoder.fit_transform(df['Genre'].values.reshape(-1, 1)).toarray()

# Create user-item interaction matrix
num_users = len(df['Id User'].unique())
num_items = genre_encoded.shape[1]
interaction_matrix = np.zeros((num_users, num_items))

for _, row in df.iterrows():
    user_idx = row['Id User'] - 174  # Adjust user index to start from 0
    item_idx = np.argmax(row['Genre'] == encoder.categories_[0])
    interaction_matrix[user_idx, item_idx] = 1



In [7]:
class MatrixFactorization(tf.keras.Model):
    def __init__(self, num_users, num_items, latent_dim):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = tf.keras.layers.Embedding(num_users, latent_dim)
        self.item_embedding = tf.keras.layers.Embedding(num_items, latent_dim)

    def call(self, inputs):
        user_indices, item_indices = inputs
        user_emb = self.user_embedding(user_indices)
        item_emb = self.item_embedding(item_indices)
        dot_product = tf.reduce_sum(tf.multiply(user_emb, item_emb), axis=1)
        return dot_product


Train the model

In [8]:
latent_dim = 16  # Dimensionality of embeddings

model = MatrixFactorization(num_users, num_items, latent_dim)
model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
# Find the minimum user index
min_user_index = np.min(df['Id User'])

# Adjust user indices to start from 0 and be within range
user_indices = np.array(df['Id User'] - min_user_index)
num_users = len(np.unique(user_indices))
max_user_index = np.max(user_indices)

if max_user_index >= num_users:
    user_indices = np.where(user_indices >= num_users, num_users - 1, user_indices)

item_indices = np.argmax(interaction_matrix, axis=1)
ratings = np.ones_like(user_indices)  # Binary interaction, set ratings to 1

model.fit([user_indices, item_indices], ratings, epochs=50, batch_size=32)





Epoch 1/50
23/23 [==============================] - 1s 2ms/step - loss: 0.9990
Epoch 2/50
23/23 [==============================] - 0s 2ms/step - loss: 0.9918
Epoch 3/50
23/23 [==============================] - 0s 2ms/step - loss: 0.9797
Epoch 4/50
23/23 [==============================] - 0s 2ms/step - loss: 0.9592
Epoch 5/50
23/23 [==============================] - 0s 3ms/step - loss: 0.9259
Epoch 6/50
23/23 [==============================] - 0s 2ms/step - loss: 0.8803
Epoch 7/50
23/23 [==============================] - 0s 2ms/step - loss: 0.8237
Epoch 8/50
23/23 [==============================] - 0s 2ms/step - loss: 0.7605
Epoch 9/50
23/23 [==============================] - 0s 2ms/step - loss: 0.6951
Epoch 10/50
23/23 [==============================] - 0s 2ms/step - loss: 0.6316
Epoch 11/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5704
Epoch 12/50
23/23 [==============================] - 0s 2ms/step - loss: 0.5126
Epoch 13/50
23/23 [==============================

Testing the recommendations from the model

In [9]:
user_id = 1  # User ID for which you want to generate recommendations

# Get the user embedding
user_emb = model.user_embedding(np.array([user_id - 1]))

# Compute the dot product between user embedding and item embeddings
item_embeddings = model.item_embedding.weights[0]
recommendations = tf.matmul(user_emb, item_embeddings, transpose_b=True)
recommendations = tf.squeeze(recommendations).numpy()

# Sort the recommendations by their scores
sorted_indices = np.argsort(recommendations)[::-1]
top_indices = sorted_indices  # Select the top 3 indices
top_items = df['Nama Produk'].iloc[top_indices]

print("Recommendations for User", user_id)
for item in top_items:
    print("- Item:", item)


Recommendations for User 1
- Item: Mascara Waterproof
- Item: Kemeja Casual
- Item: Palet Eyeshadow
- Item: Mainan Edukatif
- Item: Lipstik Matte
- Item: Tas Bayi
- Item: Raket Badminton
- Item: Buku Anak-Anak
- Item: Soto Betawi
- Item: Bros Unik
- Item: Kursi Makan Bayi
- Item: Robot Transformers
- Item: Piano Digital
- Item: Action Figure
- Item: Blus Wanita
- Item: Kalung Etnik
- Item: Dress Midi
- Item: Lukisan Abstrak
- Item: Celana Olahraga
- Item: Gelang Kalung Set
- Item: Patung Kayu
- Item: Vas Hias
- Item: Buku Inspiratif
- Item: Meja Rias
- Item: Kursi Sofa
- Item: Gelang Kaki
- Item: Keyboard Elektrik
- Item: Buku Biografi
- Item: Gorden Cantik
- Item: Gitar Akustik
- Item: Kerajinan Kain Flanel
- Item: Smart TV
- Item: Potret Lukisan
- Item: Cincin Wanita
- Item: Novel Bestseller
- Item: Raket Tenis
- Item: Celana Panjang
- Item: Kaos Oversize
- Item: Vas Tanaman
- Item: Smartphone Samsung
- Item: Vas Bunga
- Item: Kopi Arabika
- Item: Stroller Bayi
- Item: Headphone Blue

In [12]:
model_dir = 'C:\\Users\\darry\\Downloads\\model_bangkit'


# Save the model in the tf_saved_model format
tf.saved_model.save(model, model_dir)

In [13]:
model.summary()


Model: "matrix_factorization"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  11680     
                                                                 
 embedding_1 (Embedding)     multiple                  1136      
                                                                 
Total params: 12,816
Trainable params: 12,816
Non-trainable params: 0
_________________________________________________________________
